# 멀티 쓰레딩

### 1. concurrent.futures.ThreadPoolExecutor

* 동기적 : 어떤 요청을 보냈을때 그에 대한 응답을 받을때 까지 대기하였다가 응답이오면 작업 시작하는 방식
* 비동기적 : 어떤 요청을 보냈을때 그에 대한 응답을 기다리는 동안 다른 작업 실시

In [9]:
import time
start = time.time()
print(pow(2,3))
end = time.time()
print("{:.2f}".format(end - start))

8
0.00


concurrent.futures.ThreadPoolExecutor 를 excutor 로 열어준 뒤,
submit을 이용하여 처리하고 싶은 함수와 인자들을 보낸다.   
submit 외에도 map과 shutdown 등의 메서드가 존재하는데 with 구문을 이용하면 shutdown 은 사실상 필요없다. 

* sbmit  
콜러블 fn 이 fn(*args **kwargs) 처럼 실행되도록 예약하고, 콜러블 객체의 실행을 나타내는 Future 객체를 반환합니다.

In [10]:
from concurrent.futures import ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=1) as executor:
    future = executor.submit(pow, 2, 3)
    print(future.result())

8


ThreadPoolExecutor 는 스레드 풀을 사용하여 호출을 비동기적으로 실행하는 Executor 서브 클래스입니다.  
이때 동기적인 작업을 쓰레드로 진행한다면 교착상태가 발생할 수 있습니다.   
동기적 비동기적 작업은 내가 직접 정의하는게 아닌 해야하는 작업에 따라 달라집니다.  

Future와 관련된 콜러블 객체가 다른 Future 의 결과를 기다릴 때 교착 상태가 발생할 수 있습니다. 예를 들면:  

In [19]:
import time
def wait_on_b():
    time.sleep(5)
    print(b.result())  # b will never complete because it is waiting on a.
    return 5

def wait_on_a():
    time.sleep(5)
    print(a.result())  # a will never complete because it is waiting on b.
    return 6


executor = ThreadPoolExecutor(max_workers=2)
a = executor.submit(wait_on_b)
b = executor.submit(wait_on_a)
# a.result(), b.result()

위 코드의 a는 b가 끝날때 까지 기다리고 , b는 a가 끝날때 까지 기다린다. 즉 둘이 비동기식으로 진행된다면 교착상태에 빠지게 된다.

In [24]:
def wait_on_future():
    f = executor.submit(pow, 5, 2)
    # This will never complete because there is only one worker thread and
    # it is executing this function.
    print(f.result())

executor = ThreadPoolExecutor(max_workers=1)
executor.submit(wait_on_future)

<Future at 0x10772b220 state=running>

마찬가지로 위의 코드도 절대 끝나지 않는다. 
하나의 Thread 만 작동하여 그 thread 에 wait_on_future 만 할당되고,  그 함수 안에 존재하는 pow 함수를 할당할 thread수가 부족하기 때문이다.

### CONCURRENT.FUTURES 모듈 퀵 가이드

먼저 쓰레드 세개를 가진 쓰레드 풀을 만들고 함수를 넣는다. 그런다음 풀을 작업에 제출한다.
풀에 제출된 작업은 future.result() 값으로 남는다.

In [4]:
from concurrent.futures import ThreadPoolExecutor
import time 

def wait_5second(message):
    time.sleep(5)
    print('finish')
    return message

pool = ThreadPoolExecutor(3)
future = pool.submit(wait_5second, 'hello')
print(future.done())
time.sleep(6)
print(future.done())
print(future.result())

False
finish
True
hello


future 를 다루는 두개의 함수가 있음   

1. as_complete() 
as_completed() 함수는 future 객체를 반복하여 가져오고 future가 문제를 해결하기 시작되면 즉시 값을 yielding 하기 시작합니다. 앞서 언급 한 map 메소드와 as_completed의 주요 차이점은 map이 iterable을 전달한 순서대로 결과를 리턴한다는 점이다. 첫 번째 항목에 대한 결과는 map메서드의 첫 번째 결과입니다. 반면, as_completed 함수의 첫 번째 결과는 가장 먼저 완료된 future의 결과입니다.

2. wait()
함수는 두 세트를 포함하는 명명된 튜플을 반환합니다. 하나의 세트에는 완료된 futures (결과 또는 예외가 있음)과 완료되지 않은 것을 포함하는 다른 세트가 포함됩니다.
여기에 예제가 있습니다.


In [14]:
from concurrent.futures import ThreadPoolExecutor, wait, as_completed
from time import sleep
from random import randint
import os
import time

def return_after_5_secs(num):
    # sleep(randint(1, 7))
    os.makedirs(os.path.dirname('./test_txt/'),exist_ok=True)
    with open('./test_txt/{}'.format(num), 'w') as txt:
        txt.write('the number is %d'%num)
    return "Return of {}".format(num)
 
pool = ThreadPoolExecutor()
futures = []

start = time.time()
for x in range(100000):
    futures.append(pool.submit(return_after_5_secs, x))
 
for x in as_completed(futures):
    x.result()
end = time.time()
end - start

15.713350296020508

In [16]:
start = time.time()
for x in range(100000):
    return_after_5_secs(x)
end = time.time() 
end - start

14.709906101226807